# Overview

- [ ] Make map interactive with time periods,tags, pager codes.

# Data

In [ ]:
# Example

# check if any empty values
df["location_of_shout"].tolist()
# df["location_of_shout"].isnull().sum()
simple_loc=[
 'Ardlui',
 'Duckbay',
 'TaRBet',
 'Balmaha',
 'luss']

In [1]:
import pandas as pd 
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd


In [ ]:
#data
data = pd.read_csv('companies.csv')
print(data.head())

In [ ]:

# Initialize geocoder
geolocator = Nominatim(user_agent="my_geocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to geocode a location
def geocode_location(location):
    try:
        # Attempt to geocode the location
        geocoded_location = geocode(f"{location}, BC, Canada",timeout = 5)
        return geocoded_location.latitude, geocoded_location.longitude
    except:
        # Return NA values if geocoding fails
        return pd.NA, pd.NA

# Apply geocoding to each location in the DataFrame
expanded_df['coords'] = expanded_df['Location'].apply(geocode_location)
expanded_df[['latitude', 'longitude']] = pd.DataFrame(expanded_df['coords'].tolist(), index=expanded_df.index)


In [6]:
# Group by location and determine number of call outs per year.
aggregated_df = expanded_df.groupby('Location').agg({
    'min_cost': 'mean',
    'max_cost': 'mean',
    'latitude': 'first',  # Since all latitudes for the same location should be equal
    'longitude': 'first'  # Ditto for longitudes
}).reset_index()

expanded_df = expanded_df[expanded_df['Location'] != 'Su']


In [7]:
# Normalize cost data to range from 0 to 1 for heat map intensity
import folium
from folium.plugins import HeatMap

# Create a map centered around Loch lomond
map_center = aggregated_df[['latitude', 'longitude']].dropna().mean().tolist()
map = folium.Map(location=map_center, zoom_start=6)

# List of points for the heat map: lat, lng, and the weight
heat_data = [
    (row['latitude'], row['longitude'], row['cost_weight']) 
    for index, row in aggregated_df.iterrows()
    if pd.notna(row['latitude']) and pd.notna(row['longitude'])
]


In [ ]:
# Add the heat map layer
HeatMap(heat_data).add_to(map)

In [ ]:
# Save to an HTML file
map.save('heat_map.html')

# Later


In [ ]:

# Assume 'expanded_df' is already loaded and includes latitude and longitude
# Calculate the average cost
import plotly.graph_objects as go

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=expanded_df['longitude'],
    y=expanded_df['latitude'],
    z=expanded_df['average_cost'], #TODO pager color?
    mode='markers',
    marker=dict(
        size=10,
        color=expanded_df['average_cost'],    # Set color according to average cost
        colorscale='Viridis',                 # Color scale for visualization
        opacity=0.8
    ),
    text=expanded_df['Company']              # Hover text
)])

# Update plot layout
fig.update_layout(
    title='3D Plot of Companies by Location and Average Cost',
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Average Cost'
    ),
    margin=dict(l=0, r=0, b=0, t=0)          # Tight layout
)

# Show the plot
fig.show()

# Optionally, save the plot to an HTML file
fig.write_html('3D_scatter_plot.html')

expanded_df.to_csv('processed_companies.csv', index=False)

